In [10]:
import math, random
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [89]:
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self._prev = set(_children)
    self.label = label
    self._backward = lambda: None
    self.grad = 0.0
    self._op = _op

  def __repr__(self):
    return f'Value:{self.data}'

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    self = self if isinstance(self, Value) else Value(self)
    out = Value(self.data + other.data, (self, other), '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __sub__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    self = self if isinstance(self, Value) else Value(self)
    out = Value(self.data - other.data, (self, other), '-')
    def _backward():
      self.grad += out.grad * 1.0
      other.grad += out.grad * 1.0
    out._backward = _backward
    return out

  def __rsub__(self, other):
    return self * other
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), 'only int/float support'
    out = Value(self.data**other, (self,), f'**{other}')
    def _backward():
      self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward
    return out

  def __truediv__(self, other):
    out = Value(self.data / other.data, (self, other), '/')
    def _backward():
      # dout / dself, dout / dother
      self.grad += (1 / other.data) * out.grad
      other.grad += self.data * (-1 * other.data ** -2) * out.grad
    out._backward = _backward
    return out


  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      # dout / dself, dout / dother
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = _backward
    return out

  def __rmul__(self, other):
    return self * other

  def exp(self):
    out = Value(math.exp(self.data), (self,), 'exp')
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    return out
  
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()


In [90]:
##### Single Neuron

# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias
b = Value(6.827492387982534, label='b')

# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label='x1w1'
x2w2 = x2*w2; x2w2.label='x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'

# activation
out = n.tanh()

# Custom Activation 
# e = (2 * n).exp()
# out = (e - 1) / (e + 1)

# Back propogation
out.backward()

out, w1.grad, w2.grad

(Value:0.6791272391406387, 1.0775723861144275, 0.0)

In [133]:
class Neuron:
    def __init__(self, nin):
        # nin: number of inputs to the neuron
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
    

    def __call__(self, x):
        # w*x + b
        out = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = out.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self, nin, nout):
        # nin: number of inputs to a single neuron
        # nout: number of neurons in a single layer
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        out = [layer(x) for layer in self.neurons]
        return out[0] if len(out) == 1 else out
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        # nin: number of input to a single neuron
        # nouts: list of number of neurons in each layer
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x) 
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [232]:
x = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
# single row is one training example 
# number of col in single training example is the number of input of a single neuron
y = [1.0, -1.0, -1.0, 1.0]

In [233]:
nn = MLP(3, [4, 4, 1])

In [234]:
for k in range(20):

    ypred = [nn(e) for e in x] # forward pass
    loss = sum(((orig - pred)**2 for orig, pred in zip(y, ypred)), Value(0.0))

    for p in nn.parameters():
        p.grad = 0.0
    loss.backward()

    for p in nn.parameters():
        p.data += -0.05 * p.grad 

    print(k, loss.data, ypred)


0 0.9886580389864231 [Value:-0.042787656941246636, Value:0.7491121426348154, Value:-0.026880477999085974, Value:0.651870917482532]
1 0.6234818970705979 [Value:-0.41631864796593554, Value:0.5212647955153462, Value:-0.2885503932477418, Value:0.3085163916866187]
2 0.3693916435092679 [Value:-0.3517527674000081, Value:0.39521633068638035, Value:-0.11284058218672631, Value:0.27700665895652765]
3 0.21110605882119604 [Value:-0.33711535967862505, Value:0.2153069976902008, Value:-0.10291023575214876, Value:0.2012751182163765]
4 0.12202488246747817 [Value:-0.22716123026922416, Value:0.13150254441902012, Value:-0.023234781863334993, Value:0.22932484307884649]
5 0.08037694281526947 [Value:-0.20713640622289567, Value:0.022310861673561864, Value:-0.03808701880374307, Value:0.18847561243396857]
6 0.06309557521984092 [Value:-0.12048466492087737, Value:0.009905304922580252, Value:0.017237630331072985, Value:0.2195080175613292]
7 0.05541507607638892 [Value:-0.1621162028864679, Value:-0.06012974927008599,

In [223]:
nn.layers[0].neurons[0].w[0].data, nn.layers[0].neurons[0].w[0].grad 

(-0.8526652607756128, 0.013390524647337244)

In [146]:
-0.4569054185925154 * -0.01 + -0.771491679686888

-0.7669226255009629